In [2]:
import os

from langchain_cohere import CohereEmbeddings , CohereRerank , ChatCohere
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

import cohere
import qdrant_client
from qdrant_client.models import Batch
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams



In [3]:
#Retrieveing relavent documents from cohere database
def cohereRetrival(collection,textList):
    cohere_client = cohere.Client(os.environ["COHERE_API_KEY"])
    client = qdrant_client.QdrantClient()
    result = client.search(
    collection_name=collection,
    query_vector=cohere_client.embed(
        model="embed-english-v3.0",  
        input_type="search_query",  # Input type for search queries
        texts=textList,
    ).embeddings[0],
    )
    return result

import cohere

#Generating multiple generic queries from the user prompt to then retrieve the relevant documents easily
def ragFusion(prompt,collection= "MedicalPapers"):
    co = cohere.Client(os.environ["COHERE_API_KEY"])
    queryGenerationPrompt = ChatPromptTemplate.from_template("Given the prompt: '{prompt}', generate {num_queries} questions that are better articulated. Return in the form of an list. For example: ['question 1', 'question 2', 'question 3']")
    queryGenerationChain = queryGenerationPrompt | llm
    queries = queryGenerationChain.invoke({"prompt": prompt, "num_queries": 3}).content.split('\n')
    retrievedContent = []
    for query in queries:
        ret = cohereRetrival(collection,[query])
        for doc in ret:
            for key,value in doc.payload.items():
                value = value.replace(u'\xa0',u' ')
                value = value.replace(u'\t',u'  ')
                value = value.replace(u'\r',u'')
                value = value.replace(u'\n',u'      ')
                retrievedContent.append(value)
    retrievedContent = list(set(retrievedContent))
    result = co.rerank(model="rerank-english-v3.0", query=prompt, documents=retrievedContent, top_n=5, return_documents=True)
    context = ""
    for i in result.results:
        context += i.document.text
        print(i.document)
        context += "\n\n"

In [4]:
#Creating a prompt template. Giviving the llm the role of a medical assistant and adding patient history , medical context
#and finally the user prompt is added to the query
from langchain.prompts import ChatPromptTemplate
promptTemplate = template = """You are a medical assistant that specializes in providing second opinions, diagnosing complex cases 
and suggesting treatment plans. When I describe the patient details, medical context and task, give me the appropriate treatment plan 
or second opinion based on the task given by analyzing the patient details and medical context. In your answer include how your opinion or treatment
plann is related to the patient's history.

Patient History : {patientHistory}

Medical Context : {context}

Task: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [5]:
#function to invoke the langchain
def getResponce(prompt,context = "",PatientHistory=""):
    result = chain.invoke({"context":context,"patientHistory":PatientHistory,"question":prompt})
    return result

In [6]:
generationLLM =  ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")
chain = prompt|generationLLM 

In [26]:
patientHistory = """Patient Details : Patient Information:
Name: Rajesh Kumar
Age: 60
Gender: Male
Occupation: Office manager
Medical History:
Hypertension (controlled with medication)
Previous history of lumbar strain due to heavy lifting incidents, managed conservatively
Chief Complaint:
Chronic back pain
Duration: Approximately 6 months
Symptoms:
Dull, aching pain in the lower back, occasionally radiating down the right leg
Exacerbated by prolonged sitting or standing, relieved by lying down
Severity: 7/10 on average, worsening to 8/10 during flare-ups
No associated numbness or tingling
Impact on Daily Activities:
Difficulty performing work duties due to prolonged sitting at a desk
Limited ability to participate in recreational activities, such as playing sports or gardening
Struggles with household chores, particularly activities that involve bending or lifting
Treatment History:
Nonsteroidal anti-inflammatory drugs (NSAIDs) for pain management, with minimal relief
Physical therapy sessions focusing on core strengthening exercises and flexibility, providing short-term improvement but limited long-term benefits
Occasional use of heat packs and over-the-counter topical analgesics for symptomatic relief
Current Medications:
Amlodipine 5 mg daily for hypertension
Ibuprofen 400 mg as needed for back pain
Additional Concerns:
Rajesh is concerned about the impact of his back pain on his job performance and overall quality of life.
He's interested in exploring alternative treatment options, such as acupuncture or chiropractic care.
 """
context = """ Ibuprofen is used to relieve pain from various conditions such as headache, dental pain, menstrual cramps, muscle aches, or arthritis. It is also used to reduce fever and to relieve minor aches and pain due to the common cold or flu. Ibuprofen is a nonsteroidal anti-inflammatory drug (NSAID). It works by blocking your body's production of certain natural substances that cause inflammation. This effect helps to decrease swelling, pain, or fever.If you are treating a chronic condition such as arthritis, ask your doctor about non-drug treatments and/or using other medications to treat your pain. See also Warning section.This form of ibuprofen is intended for use in children. Do not give this medication to a child younger than 2 years unless directed by the doctor.Check the ingredients on the label even if you have used the product before. The manufacturer may have changed the ingredients. Also, products with similar names may contain different ingredients meant for different purposes. Taking the wrong product could harm you.
Amlodipine is used alone or in combination with other medications to treat high blood pressure in adults and children 6 years and older. It is also used to treat certain types of angina (chest pain) and coronary artery disease (narrowing of the blood vessels that supply blood to the heart) """

question = """ I'm facing challenges in managing a case of chronic back pain in a patient. Despite thorough evaluation and treatment attempts, including physical therapy and medication, the pain persists. The patient experiences a constant dull ache localized to the lumbar region, limiting their mobility and affecting their quality of life. I'm seeking your expertise to explore alternative treatment strategies or diagnostic approaches that may offer relief. """

In [31]:
CancerPatientHistory = """Patient information

Name: John Smith

Age: 58

Contact: john.smith@email.com | (555) 123-4567

Chief complaint

John has persistent cough, chest pain, and shortness of breath for six months. He reports recent weight loss and fatigue.

Present illness

John describes a progressive cough with occasional blood-streaked sputum. Chest pain worsens during coughing, deep breathing, and movement. Shortness of breath has limited his daily activities.

Medical history

John's medical history includes hypertension, but no cancer diagnosis. He had an appendectomy in his 30s.

Family medical history

John mentions no history of lung cancer in his family, but his father passed away due to a heart attack.

Lifestyle and risk factors

John used to smoke two packs of cigarettes daily for 30 years before quitting five years ago. He has a sedentary lifestyle and admits to occasional alcohol consumption.

Medication and allergies

Current medications: Amlodipine for hypertension.

Allergies: None reported.

Social and emotional aspects

John lives with his wife and supportive family. However, he expresses anxiety and fear about the cancer diagnosis.

Review of systems

    Respiratory: Cough, sputum production, dyspnea.
    Gastrointestinal: No specific issues reported.
    Cardiovascular: No palpitations or chest discomfort other than lung-related pain.
    Neurological: No significant complaints.
    Musculoskeletal: Some discomfort due to chest pain.

Physical examination

    General: John appears fatigued but alert.
    Respiratory: Decreased breath sounds on the right side. Dullness on percussion over the right lung base.
    Cardiovascular: Regular rhythm, no murmurs.
    Other systems: Non-contributory findings.

Investigations and imaging

    Chest X-ray: Right lung mass and pleural effusion.
    CT scan: Further characterization of the lung tumor and its extent.
    Biopsy: Pending for definitive diagnosis. """

In [25]:
result = chain.invoke({"context":context , "patientHistory":patientHistory,"question":question})
to_markdown(result.content)

> ## Second Opinion and Treatment Plan for Rajesh Kumar
> 
> Based on the provided information, Rajesh Kumar is experiencing chronic lower back pain with potential radicular symptoms (pain radiating down the leg). While initial conservative management has been implemented, it seems the current approach needs further refinement.
> 
> **Here's my analysis and recommendations:**
> 
> **Possible Causes:**
> 
> * **Lumbar Disc Herniation:** Given the radiating pain and history of heavy lifting, a herniated disc pressing on a nerve root is a possibility. 
> * **Lumbar Spinal Stenosis:** This condition involves narrowing of the spinal canal, which can compress nerves and cause pain. 
> * **Degenerative Disc Disease:** Wear and tear on the spinal discs can lead to pain and instability. 
> * **Facet Joint Syndrome:** Inflammation of the facet joints in the spine can cause back pain and stiffness.
> 
> **Diagnostic Considerations:**
> 
> * **MRI Lumbar Spine:** To visualize the spinal structures and identify any disc herniations, stenosis, or other abnormalities.
> * **Electromyography (EMG) and Nerve Conduction Studies (NCS):** To assess nerve function and rule out nerve compression as the source of pain.
> 
> **Treatment Plan Modifications:**
> 
> * **Medications:**
>     * **Muscle Relaxants:** To address muscle spasms that may be contributing to the pain.
>     * **Neuropathic Pain Medications:** If nerve involvement is confirmed, medications like gabapentin or pregabalin can be considered.
>     * **Short-term Opioid Analgesics:** May be considered for acute exacerbations under careful monitoring.
> * **Interventional Procedures:**
>     * **Epidural Steroid Injections:** To deliver anti-inflammatory medication directly to the affected area and provide temporary pain relief.
>     * **Facet Joint Injections:** If facet joint syndrome is suspected, injections can help diagnose and treat the condition.
> * **Physical Therapy:**
>     * **Review and modify the current physical therapy regimen:** Focus on exercises that specifically address the identified cause of pain (e.g., McKenzie exercises for disc herniation).
>     * **Consider aquatic therapy:** This can provide a low-impact environment for exercise and pain relief.
> * **Alternative Therapies:**
>     * **Acupuncture:** May offer pain relief for some individuals.
>     * **Chiropractic Care:** Can be helpful for musculoskeletal alignment, but ensure the chiropractor is experienced in managing disc-related conditions.
> 
> **Addressing Rajesh's Concerns:**
> 
> * **Impact on Job Performance:** Discuss ergonomic adjustments to his workstation and explore options for modified duties or work-from-home arrangements if necessary.
> * **Quality of Life:** Encourage Rajesh to remain active within his pain limits and consider low-impact activities like swimming or walking.
> * **Alternative Therapies:** Discuss the potential benefits and limitations of acupuncture and chiropractic care, ensuring he seeks qualified practitioners.
> 
> **Connecting Treatment Plan to Patient History:**
> 
> * Rajesh's history of heavy lifting incidents suggests a potential for disc-related problems.
> * His age makes degenerative changes in the spine more likely.
> * His controlled hypertension indicates he is likely able to tolerate most treatment options.
> 
> **Disclaimer:** This information is intended for general knowledge and informational purposes only, and does not constitute medical advice. It is essential to consult with a qualified healthcare professional for any health concerns or before making any decisions related to your health or treatment. 


In [33]:
question = """ I'm facing challenges in managing a case of chronic back pain in a patient. Despite thorough evaluation and treatment attempts, 
including physical therapy and medication, the pain persists. 
The patient experiences a constant dull ache localized to the lumbar region, 
limiting their mobility and affecting their quality of life. I'm seeking your expertise to
explore alternative treatment strategies or diagnostic approaches that may offer relief. """

In [35]:
model2 = genai.GenerativeModel(model_name = 'gemini-1.5-pro-latest')
chat = model2.start_chat(history = []) 

In [36]:
cancerQuestion = "Suggest a preliminary treatment plan for the patient based on his personal history and symptoms"

In [37]:
result = chain.invoke({"context":context , "patientHistory":CancerPatientHistory,"question":cancerQuestion})

In [38]:
to_markdown(result.content)

> ## Preliminary Treatment Plan for John Smith
> 
> **Based on the information provided, John's symptoms and test results strongly suggest lung cancer, pending biopsy confirmation.**  His age, significant smoking history, persistent cough with blood, chest pain, weight loss, and the lung mass seen on imaging all point towards this diagnosis.  
> 
> **Here's a preliminary treatment plan, keeping in mind that definitive treatment will depend on the biopsy results and the specific type and stage of lung cancer:**
> 
> **1.  Pain Management:**
> 
> * **Non-opioid analgesics:** Given John's history of hypertension, we should avoid NSAIDs like ibuprofen as they can worsen blood pressure.  Instead, we can consider acetaminophen for pain relief. 
> * **Opioids:** If the pain is severe and not controlled by acetaminophen,  we may need to consider short-term use of opioids with careful monitoring for side effects and potential for dependence.
> * **Palliative Care Referral:**  Early involvement of palliative care can help manage pain,  symptoms, and emotional distress associated with the diagnosis and treatment. 
> 
> **2.  Symptom Management:**
> 
> * **Cough suppressants:** To address the persistent cough and improve quality of life.
> * **Oxygen therapy:** If John experiences shortness of breath, oxygen therapy can improve his oxygen levels and make breathing easier.
> * **Pulmonary rehabilitation:** This program can help improve lung function and manage breathlessness through exercise training and breathing techniques.
> 
> **3.  Emotional Support:**
> 
> * **Psychological counseling:** Addressing John's anxiety and fear about the diagnosis is crucial.  Individual or family counseling can provide coping mechanisms and emotional support. 
> * **Support groups:** Connecting with others facing similar challenges can be beneficial. 
> 
> **4.  Diagnostic Confirmation and Staging:**
> 
> * **Biopsy:** This is essential to confirm the type of lung cancer and guide treatment decisions.
> * **Additional imaging:**  Further imaging tests like PET scan or MRI might be needed to determine the stage of cancer and if it has spread to other parts of the body.
> 
> **5.  Definitive Treatment:**
> 
> * **Surgery:** If the cancer is localized, surgical removal of the tumor or part of the lung may be considered.
> * **Chemotherapy:** This is often used to kill cancer cells or shrink the tumor.
> * **Radiation therapy:** This uses high-energy beams to target and destroy cancer cells.
> * **Targeted therapy:** This treatment targets specific genetic mutations within the cancer cells.
> * **Immunotherapy:** This treatment helps boost the body's immune system to fight cancer cells. 
> 
> **The choice of definitive treatment will depend on several factors including the type and stage of lung cancer, John's overall health, and his personal preferences. A multidisciplinary team of oncologists, pulmonologists, surgeons, and other specialists will work together to develop the best treatment plan for him.** 
> 
> **It is important to note that this is a preliminary plan, and  modifications may be needed based on the biopsy results and further investigations. Open communication and shared decision-making with John and his family are crucial throughout the treatment process.**
